# Prepare biomass carbon data

In [ ]:
dir01 = '../paper_deficit/output/01_prep/'
dir02 = '../paper_deficit/output/02_dbase/'

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt

---

In [ ]:
# Load datasets
ds_esabio_agb = xr.open_mfdataset(dir01 + 'ds_prep_esabio_agb*.zarr', 
                                  engine='zarr')
ds_spawn_agbc = xr.open_zarr(dir01 + 'ds_prep_spawn_agbc.zarr')
ds_spawn_bgbc = xr.open_zarr(dir01 + 'ds_prep_spawn_bgbc.zarr')
ds_huang_agb = xr.open_zarr(dir01 + 'ds_prep_huang_agb.zarr')
ds_huang_bgb = xr.open_zarr(dir01 + 'ds_prep_huang_bgb.zarr')

In [ ]:
# Calculate mean esabio carbon of 2015-2021 using IPCC default
vars_esabio_agb = [i for i in ds_esabio_agb.data_vars 
                   if i.startswith('esabio_agb20')]

da_esabio_agbc = ds_esabio_agb[vars_esabio_agb] \
    .to_array('a') \
    .mean('a') \
    .rename('esabio_agbc') * 0.47

# Merge esabio and spawn data into a single array
da_agbc = xr.merge([da_esabio_agbc, ds_spawn_agbc.spawn_agbc]) \
    .to_array('agbc')

# Calculate arrays with min, mean, and max values for each grid cell
da_agbc_min = da_agbc.min('agbc')
da_agbc_mean = da_agbc.mean('agbc')
da_agbc_max = da_agbc.max('agbc')

# Create AGBC dataset
ds_agbc = xr.Dataset({
    'agbc_min': da_agbc_min,
    'agbc_mean': da_agbc_mean,
    'agbc_max': da_agbc_max
})

# Export AGBC dataset
ds_agbc.to_zarr(dir02 + 'ds_prep_agbc.zarr', mode='w');

In [ ]:
# Get Huang bgb/agb ratio data
ds_huang_ratio = xr.open_zarr(dir01 + "ds_prep_huang_bgbagbratio.zarr")
da_huang_ratio = ds_huang_ratio.huang_bgbagbratio

# Calculate BGBC carbon values using AGBC and Huang ratio
ds_bgbc = xr.Dataset({
    'bgbc_min': da_agbc_min * da_huang_ratio, 
    'bgbc_mean': da_agbc_mean * da_huang_ratio,
    'bgbc_max': da_agbc_max * da_huang_ratio 
})

# Export BGBC dataset
ds_bgbc.to_zarr(dir02 + 'ds_prep_bgbc.zarr', mode='w');

---

### Check

In [ ]:
ds_agbc = xr.open_zarr(dir02 + 'ds_prep_agbc.zarr')
ds_bgbc = xr.open_zarr(dir02 + 'ds_prep_bgbc.zarr')
ds_area = xr.open_zarr(dir01 + 'ds_prep_area_ha.zarr')

In [ ]:
for i in list(ds_agbc.data_vars):
    fig, ax = plt.subplots(figsize=(10, 5), ncols=1, nrows=1)
    ds_agbc[i].plot.imshow(ax=ax, robust=True)
    ax.set_title(i)

In [ ]:
for i in list(ds_bgbc.data_vars):
    fig, ax = plt.subplots(figsize=(10, 5), ncols=1, nrows=1)
    ds_bgbc[i].plot.imshow(ax=ax, robust=True)
    ax.set_title(i)

In [ ]:
def get_pgc(da):
    v_pgc = ((da * ds_area.area_ha).sum(['lat', 'lon']) * 0.000000001) \
        .compute().item() 
    return round(v_pgc, 1)

In [ ]:
print('Spawn AGBC: ' + str(get_pgc(ds_spawn_agbc.spawn_agbc)))
print('Spawn BGBC: ' + str(get_pgc(ds_spawn_bgbc.spawn_bgbc)))
print('ESACCI: ' + str(get_pgc(da_esabio_agbc)))
print('Huang AGBC: ' + str(get_pgc(ds_huang_agb.huang_agb * 0.47)))
print('Huang BGBC: ' + str(get_pgc(ds_huang_bgb.huang_bgb * 0.47)))

In [ ]:
print('Max Scenario')
print('AGBC: ' + str(get_pgc(ds_agbc.agbc_max)))
print('BGBC: ' + str(get_pgc(ds_bgbc.bgbc_max)))
print('AGBC + BGBC: ' + str(get_pgc(ds_agbc.agbc_max + ds_bgbc.bgbc_max)))

In [ ]:
print('Mean Scenario')
print('AGBC: ' + str(get_pgc(ds_agbc.agbc_mean)))
print('BGBC: ' + str(get_pgc(ds_bgbc.bgbc_mean)))
print('AGBC + BGBC: ' + str(get_pgc(ds_agbc.agbc_mean + ds_bgbc.bgbc_mean)))

In [ ]:
print('Min Scenario')
print('AGBC: ' + str(get_pgc(ds_agbc.agbc_min)))
print('BGBC: ' + str(get_pgc(ds_bgbc.bgbc_min)))
print('AGBC + BGBC: ' + str(get_pgc(ds_agbc.agbc_min + ds_bgbc.bgbc_min)))

---